# Visualizations - Heatmaps

In [4]:
from math import pi
import pandas as pd

from bokeh.models import BasicTicker, PrintfTickFormatter
from bokeh.palettes import Viridis256
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import linear_cmap, transform, LinearColorMapper

In [24]:
def generate_hmap(df, legend=False, export=False, orient_h=False, region=None, start_yr=2013, end_yr=2022):
    raw_df_ll_min = df["Life Ladder"].min()
    raw_df_ll_max = df["Life Ladder"].max()
    if region:
        df = df[(df["Year"] >= start_yr) & (df["Year"] <= end_yr) & (df["region"] == region)].copy()
    else:
        df = df[(df["Year"] >= start_yr) & (df["Year"] <= end_yr)].copy()

    df["Year"] = df["Year"].astype(str)
    df.rename(columns={"Life Ladder": "LL"}, inplace=True)
    df = df.pivot(index="Year", columns="shortname", values="LL")

    df.columns.name = "country"
    years = list(df.index)
    countries = list(reversed(df.columns))
    if orient_h:
        yr, y_colname = years, "Year"
        xr, x_colname = sorted(countries), "country"
        w = len(countries)*10 + 50
        h = (end_yr-start_yr)*25
    else:
        yr, y_colname = countries, "country"
        xr, x_colname = years, "Year"
        w = (end_yr-start_yr)*40+20
        h = len(countries)*10

    # reshape to 1D array
    df = pd.DataFrame(df.stack(), columns=['LL']).reset_index()
    
    color_mapper = LinearColorMapper(palette=Viridis256, low=raw_df_ll_min, high=raw_df_ll_max)

    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

    p = figure(title=f"Average Life Evaluation Scores {'- ' + region if region else ''}",
               x_range=xr, y_range=yr,
               x_axis_location="above", width=w, height=h, min_height=100,
               tools=TOOLS, toolbar_location='below',
               tooltips=[('Country', '@country'), ('Year', '@Year'), ('Score', '@LL')]
              )

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "7px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = pi / 3

    r = p.rect(x=x_colname, y=y_colname, width=1, height=1, source=df,
               fill_color=transform('LL', color_mapper),
               line_color=None)

    if legend:
        p.add_layout(r.construct_color_bar(
            major_label_text_font_size="7px",
            ticker=BasicTicker(desired_num_ticks=7),
            formatter=PrintfTickFormatter(format="%d"),
            label_standoff=6,
            border_line_color=None,
            padding=5,
        ), 'below')

    if not export:
        output_notebook()
    show(p)

In [14]:
df = pd.read_csv("./data/processed/WHR_LL_Region_20052022.csv", index_col=0)
sorted(df["region"].unique())

['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']

In [7]:
generate_hmap(df=df, export=True, orient_h=False)

In [8]:
for region in sorted(df["region"].unique()):
    generate_hmap(export=True, orient_h=False, df=df, region=region, start_yr=2013)

In [9]:
generate_hmap(df=df, export=True, orient_h=True)

In [10]:
for region in sorted(df["region"].unique()):
    generate_hmap(export=True, orient_h=True, df=df, region=region, start_yr=2013)

In [27]:
generate_hmap(df=df, export=False, orient_h=False, legend=True)

Loading BokehJS ...

In [18]:
generate_hmap(df=df, export=False, orient_h=True, legend=True)

Loading BokehJS ...

In [21]:
for region in sorted(df["region"].unique()):
    generate_hmap(export=True, orient_h=False, df=df, region=region, start_yr=2013, legend=True)